In [1]:
import pandas as pd
import numpy as np
import glob
pd.options.display.max_columns = 30
pd.options.display.max_rows = 500

### First step: preliminary cleaning of the database
#### *Preliminary note: the records for 2020 have been extracted in february 2020. Therefore, they are obviously incomplete, which explains some of the discrepancies that can be observed throughout this analysis, regarding the year 2020.*
* Concatenate all csvs in a single dataframe
* Removing all unnecessary info for the analysis

In [2]:
# Concatenating all the csvs
csv_files = glob.glob("../data/final-data/*.csv")
list_data = []
for filename in csv_files:
    data = pd.read_csv(filename, encoding = "latin1", low_memory=False)
    list_data.append(data)
df = pd.concat(list_data,ignore_index=True)

# Sorting by year
df.sort_values(by=["EXERCICE"], ascending = False, inplace=True)

# Removing random trailing whitespaces
for column in df:
    if df[column].dtype == "O":
        df[column] = df[column].str.strip()

# Removal of non-course entries
df.dropna(subset=["CODE_EPR"], inplace=True)

# Removal of personal identifiable information
df.drop(columns=["Preelev", "Nomelev", "DATE_NAISELEV", "CIVILELEV", "CODE_PAYS_NAT", "DOUBLENAT", "RKG_NAT"], inplace=True)

# Removal of mandatory courses
df = df[df["OBLIG"] != 1]
df = df[df["OBLIG"] != 2]
df = df[df["OBLIG"] != 3]

# Removal of non-PGE students
df = df[df["CODE_SITE"] == "TARGET CAMPUS"]

df.shape
try:
    df.drop(columns=["Unnamed: 0"], inplace=True)
except:
    pass


In [3]:
df.shape

(591319, 23)

#### Result: the dataframe contains approximately 591k records (down from initial >1,500k)

### Step two: deep cleaning of the dataset
*See comments for details*

In [4]:
# Removing courses with few students or targeted at a very specific audience
df = df[~df["CODE_EPR"].astype(str).str.contains("^[5OBCILSVX]+")]
df = df[~df["CODE_EPR"].astype(str).str.startswith("EM")]
df = df[~df["CODE_EPR"].astype(str).str.startswith("HUMA")]
df = df[~df["CODE_EPR"].astype(str).str.startswith("EI")]
df = df[~df["CODE_EPR"].astype(str).str.startswith("RL")]
df = df[~df["CODE_EPR"].astype(str).str.startswith("ERM")]
df = df[~df["CODE_EPR"].astype(str).str.startswith("GE")]
df = df[~df["CODE_EPR"].astype(str).str.startswith("POL")]

list_of_wrong_courses = ["TUMMT","TUMESS", "GA11", "XAPEC", "EARN1", "EARN2",
                         "EARN3", "EXPEML", "MKCONF", "MKSUIV","RL1M&I",
                         "RL3F&S","MAKRAC"]
df=df[~df["CODE_EPR"].str.contains("|".join(list_of_wrong_courses))]

# Removal of courses with fewer than 50 students in total
df = df.groupby(by="CODE_EPR").filter(lambda x: len(x) >= 50)
# Removal of students with less than 10 courses in total
df = df.groupby("CODELEV").filter(lambda x: len(x) >= 10)
# Replacing SH prefix (Shanghai) with standard course code
df["CODE_EPR"] = df["CODE_EPR"].str.replace("SH", "")

# Removing non-PGE students
df=df[df["LIBETUDE"].str.contains("^.*(Année|Auditeur|AUDITEUR).*$")]
df=df[~df["LIBETUDE"].str.contains("M.S.")]
df=df[~df["LIBETUDE"].str.contains("Report")]
df=df[~df["LIBETUDE"].str.contains("MBA")]
df=df[~df["LIBETUDE"].str.contains("Année 1")]
df=df[~df["LIBETUDE"].str.contains("exch")]
df=df[~df["LIBETUDE"].str.contains("Auditeur PM")]

try:
    # Removing useless fields
    col_drop_list = ["date_deb", "date_fin", "pays_stage", "exe_bul", 
                     "CODE_SITE", "CODE_PGM", "CODE_ETUDE", "CODE_CENTRE",
                     "Duree_de_stage"]
    df.drop(columns = col_drop_list, inplace=True)
except:
    pass
else:
    print("operations performed")

# Printing our final list of courses
print(sorted(df["CODE_EPR"].unique()))

# Making a small df that lists the courses
df_ldc = (df.groupby(["CODE_EPR", "libepr"])["CODELEV"].nunique())

/home/clemdep/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


operations performed
['DR10', 'DR101', 'DR102', 'DR104', 'DR107', 'DR108', 'DR109', 'DR110', 'DR111', 'DR13', 'DR20', 'DR200', 'DR21', 'DR22', 'DR23', 'DR24', 'DR25', 'DR26', 'DR261', 'DR27', 'DR28', 'DR29', 'DR30', 'DR31', 'DR311', 'DR32', 'DR34', 'DR34A', 'DR35', 'DR40', 'DR99', 'EC100', 'EC101', 'EC102', 'EC103', 'EC105', 'EC106', 'EC107', 'EC113', 'EC115', 'EC116', 'EC116A', 'EC117', 'EC118', 'EC139', 'EC203', 'EC204', 'EC21', 'EC23', 'EC33', 'EC34', 'EC35', 'EC37', 'EC38', 'EC40', 'EC42', 'ECETHI', 'GC22', 'GC25', 'GC31', 'GC32', 'GC33', 'GC34', 'GC35', 'GC36', 'GC37', 'GC38', 'GC39', 'GC41', 'GC41A', 'GC43', 'GCC1', 'GF100', 'GF101', 'GF102', 'GF104', 'GF105', 'GF106', 'GF106A', 'GF106S', 'GF107', 'GF109', 'GF109A', 'GF11', 'GF110', 'GF11A', 'GF12', 'GF120', 'GF120A', 'GF121A', 'GF122A', 'GF123', 'GF12A', 'GF13', 'GF130A', 'GF131A', 'GF132', 'GF133', 'GF14', 'GF140', 'GF150', 'GF160', 'GF21', 'GF23', 'GF24', 'GF25', 'GF25A', 'GF26', 'GF26A', 'GF27', 'GF27A', 'GF29', 'GF30', 'GF31

In [5]:
print(df.shape)
print(df_ldc.unique().shape)

(111655, 14)
(311,)


#### Result: a dataframe containing approximately 112k records and 311 different courses.

In [6]:
# Saving the dataframe
df.to_csv("../data/data-cleaned.csv")